<a href="https://colab.research.google.com/github/jorgebecerrilgi/limpieza/blob/main/GameOfLife.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M1. Actividad
Jorge Claudio González Becerril
<br>A01412375
<br>10/Nov/2022

## Librerías e Imports
Nuestro programa utiliza tres librerías: mesa, matplotlib, numpy, y pandas.

### Mesa
La librería principal. Nos permite construir nuestros modelos basados en agentes.
### Matplotplit
Es una librería de visualización. Esta nos permite que al correr nuestro modelo lo podamos ver en una animación 2D.

In [ ]:
!pip install mesa matplotlib numpy pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from mesa import Agent, Model 
# SingleGrid fuerza a un solo objeto por celda.
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
# DataCollector permite obtener el grid completo en cada generación.
from mesa.datacollection import DataCollector

# mathplotlib para graficar la evolución del autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# numpy para cálculos númericos.
import numpy as np
import pandas as pd

import time
import datetime

import random

## Modelo


Aquí definimos el comportamiento de nuestros dos agentes, la aspiradora y el piso. Así como la habitación que será nuestro modelo, y  tendrá toda la información necesaria para empezar la simulación.

In [ ]:
def obtener_habitacion(modelo):
    habitacion = np.zeros((modelo.grid.width, modelo.grid.height))
    for celda in modelo.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:
            if isinstance(contenido, Aspiradora):
                habitacion[x][y] = 2
            else:
                habitacion[x][y] = contenido.estado
    return habitacion

class Aspiradora(Agent):

    def __init__(self, unique_id: int, model: Model) -> None:
        super().__init__(unique_id, model)
        self.nueva_posicion = None
        self.movimientos = 0
    
    def step(self) -> None:
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore = True,
            include_center = True
        )

        # Definir el sig. estado del piso.
        for vecino in vecinos:
            if isinstance(vecino, Piso) and vecino.pos == self.pos:
                vecino.siguiente_estado = vecino.estado
                if vecino.siguiente_estado == vecino.SUCIO:
                    vecino.siguiente_estado = vecino.LIMPIO
                else:
                    vecindario = self.model.grid.get_neighborhood(
                        self.pos,
                        moore = True,
                        include_center=False
                    )
                    nueva_posicion = self.random.choice(vecindario)
                    self.nueva_posicion = nueva_posicion
                break
    
    def advance(self) -> None:

        # Actualizar el estado del piso
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore = False,
            include_center = True
        )

        for vecino in vecinos:
            if isinstance(vecino, Piso) and vecino.pos == self.pos:
                vecino.estado = vecino.siguiente_estado
                break
        
        if self.pos != self.nueva_posicion:
            self.movimientos = self.movimientos + 1

        # Movemos la aspiradora a nueva posición.
        self.model.grid.move_agent(self, self.nueva_posicion)

class Piso(Agent):

    SUCIO = 1
    LIMPIO = 0

    def __init__(self, pos: int, model: Model, estado=LIMPIO) -> None:
        super().__init__(pos, model)
        self.x, self.y = pos
        self.estado = estado
        self.siguiente_estado = None

class Habitacion(Model):

    def __init__(self, m, n, num_agentes, por_celdas_sucias) -> None:
        self.num_agentes = num_agentes
        self.por_celdas_sucias = por_celdas_sucias
        self.por_celdas_limpias = 1 - por_celdas_sucias
        self.grid = MultiGrid(m, n, True)
        self.schedule = SimultaneousActivation(self)

        # Posicionar celdas sucias de forma aleatoria.
        celdas_sucias = int((m * n) * por_celdas_sucias)
        lista_celdas_vacias = list(self.grid.empties)
        for celdas in range(celdas_sucias):
            celda_vacia = random.choice(lista_celdas_vacias)
            piso = Piso(celda_vacia, self)
            piso.estado = piso.SUCIO
            self.grid.place_agent(piso, celda_vacia)
            self.schedule.add(piso)
            lista_celdas_vacias.remove(celda_vacia)
        
        # Posicionar celdas limpias.
        lista_celdas_vacias = list(self.grid.empties)
        for celdas in lista_celdas_vacias:
            piso = Piso(celdas, self)
            self.grid.place_agent(piso, celdas)
            self.schedule.add(piso)

        for i in range(num_agentes):
            aspiradora = Aspiradora(i, self)
            self.grid.place_agent(aspiradora, (1, 1))
            self.schedule.add(aspiradora)
        
        self.colectordatos = DataCollector(
            model_reporters = {'Habitacion': obtener_habitacion},
            agent_reporters = {'Movimientos': lambda a: getattr(a, 'movimientos', None)}
        )
    
    def step(self) -> None:
        self.colectordatos.collect(self)
        self.schedule.step()
    
    def todasceldaslimpias(self):
        celdas_limpias = 0
        for celda in self.grid.coord_iter():
            contenido_celda, x, y = celda
            for contenido in contenido_celda:
                if isinstance(contenido, Piso) and contenido.estado == contenido.LIMPIO:
                    celdas_limpias = celdas_limpias + 1
        
        self.por_celdas_limpias = celdas_limpias / (self.grid.width * self.grid.height)
        return self.por_celdas_limpias == 1

## Simulación

Aquí es donde vamos a generar el modelo de nuestra simulación, y la vamos a ejecutar. Definimos el tamaño de nuestra habitación, la cantidad de agentes y celdas sucias, así como el tiempo máximo de ejecución.

In [ ]:
M = 15
N = 15

NUM_AGENTES = 500

PORCENTAJE_CELDAS_SUCIAS = 0.5

# En segundos.
TIEMPO_MAXIMO_EJECUCION = 0.06

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds = TIEMPO_MAXIMO_EJECUCION))
modelo = Habitacion(M, N, NUM_AGENTES, PORCENTAJE_CELDAS_SUCIAS)

while ((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and not modelo.todasceldaslimpias()):
    modelo.step()

tiempo_ejecucion = str(datetime.timedelta(seconds = (time.time() - start_time)))

In [ ]:
todas_habitaciones = modelo.colectordatos.get_model_vars_dataframe()

## Visualización

Con ayuda de matplotlib, utilizamos los resultados generados por nuestra simulación para obtener una representación visual, que podemos reproducir.

In [ ]:
%%capture

fig, axs = plt.subplots(figsize = (7, 7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(todas_habitaciones.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(todas_habitaciones.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames = len(todas_habitaciones))

In [ ]:
anim

## Análisis
En esta sección del programa, podemos leer diferente información sobre la ejecución de nuestra simulación, como:


*   Tiempo necesario hasta que todas las celdas se encuentren limpias
*   Cantidad de celdas limpias
*   Movimientos totales realizados



In [ ]:
movimientos = modelo.colectordatos.get_agent_vars_dataframe()
print(f'Tiempo necesario hasta que todas las celdas se encuentren limpias: {tiempo_ejecucion} / {tiempo_inicio}')
print(f'Celdas limpias: {modelo.por_celdas_limpias * 100}%')
print(f'Número de movimientos realizados por los agentes: {movimientos.tail()["Movimientos"].sum()}')

Tiempo necesario hasta que todas las celdas se encuentren limpias: 0:00:00.067213 / 0:00:00.060000
Celdas limpias: 58.666666666666664%
Número de movimientos realizados por los agentes: 20.0
